In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import os, glob
import math

lcs = pd.read_csv('/content/drive/Othercomputers/My Mac/Work/Fermi_GRBs/lcs.csv')


In [12]:
if torch.cuda.is_available():
       device = torch.device("cuda")
       print("GPU is available")
else:
       device = torch.device("cpu")
       print("GPU is not available, using CPU instead")


GPU is available


In [13]:
channels = ['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb', 'b1', 'b2']

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Fill missing channels with zeros
for channel in channels:
  lcs[channel].fillna(0.0, inplace=True)

time_series_list = []
grouped = lcs.groupby('burst')
for burst, group in grouped:
    time_series_data = group[channels].values
    time_series_tensor = torch.tensor(time_series_data, dtype=torch.float32)
    time_series_list.append(time_series_tensor)

class TimeSeriesDataset(Dataset):
    def __init__(self, time_series_list):
        self.time_series_list = time_series_list

    def __len__(self):
        return len(self.time_series_list)

    def __getitem__(self, idx):
        return self.time_series_list[idx]

# Pad sequences to the same length
def collate_fn(batch):
    batch = nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.0)
    return batch

# Define the Positional Encoding Class
class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

# Define the Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        self.positional_encoding = PositionalEncoding(model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, src):
        src = self.embedding(src)
        src = self.positional_encoding(src)
        output = self.transformer_encoder(src)
        return output

# Define the Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, model_dim, output_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        decoder_layer = nn.TransformerDecoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(model_dim, output_dim)

    def forward(self, tgt, memory):
        output = self.transformer_decoder(tgt, memory)
        output = self.output_layer(output)
        return output

# Define the Autoencoder
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerAutoencoder, self).__init__()
        self.encoder = TransformerEncoder(input_dim, model_dim, num_heads, num_layers, dropout)
        self.decoder = TransformerDecoder(model_dim, input_dim, num_heads, num_layers, dropout)

    def forward(self, src):
        memory = self.encoder(src)
        tgt = self.encoder.embedding(src)
        tgt = self.encoder.positional_encoding(tgt)
        output = self.decoder(tgt, memory)
        return output, memory


<ipython-input-14-09538db39e4d>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lcs[channel].fillna(0.0, inplace=True)


In [21]:
# Parameters
input_dim = 14
model_dim = 64
num_heads = 8
num_layers = 4
batch_size = 16
num_epochs = 50
learning_rate = 0.001

# Initialize the autoencoder
autoencoder = TransformerAutoencoder(input_dim, model_dim, num_heads, num_layers)

# Create the dataset and dataloader
dataset = TimeSeriesDataset(time_series_list)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)


In [22]:
autoencoder = autoencoder.to(device)

In [23]:
# Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        # Original shape of batch: (batch_size, sequence_length, input_dim)
        # Permute to shape: (sequence_length, batch_size, input_dim)
        batch = batch.permute(1, 0, 2)
        batch = batch.to(device)

        # Forward pass
        reconstructed, latent = autoencoder(batch)

        # Compute the loss
        loss = criterion(reconstructed, batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50], Loss: 1316924.3750
Epoch [2/50], Loss: 2618347.5000
Epoch [3/50], Loss: 1323944.6250
Epoch [4/50], Loss: 2073745.1250
Epoch [5/50], Loss: 1306518.8750
Epoch [6/50], Loss: 1224891.6250
Epoch [7/50], Loss: 1430387.0000
Epoch [8/50], Loss: 1604286.2500
Epoch [9/50], Loss: 1289956.1250
Epoch [10/50], Loss: 1163014.0000
Epoch [11/50], Loss: 1997914.7500
Epoch [12/50], Loss: 526818.7500
Epoch [13/50], Loss: 1345770.8750
Epoch [14/50], Loss: 1987190.8750
Epoch [15/50], Loss: 1208355.2500
Epoch [16/50], Loss: 1096158.8750
Epoch [17/50], Loss: 724359.8125
Epoch [18/50], Loss: 1298631.7500
Epoch [19/50], Loss: 1610448.0000
Epoch [20/50], Loss: 1109384.1250
Epoch [21/50], Loss: 1428247.0000
Epoch [22/50], Loss: 1249997.1250
Epoch [23/50], Loss: 2344485.5000
Epoch [24/50], Loss: 1264824.7500
Epoch [25/50], Loss: 1144072.7500
Epoch [26/50], Loss: 1722788.8750
Epoch [27/50], Loss: 1270925.6250
Epoch [28/50], Loss: 1089369.8750
Epoch [29/50], Loss: 1509540.3750
Epoch [30/50], Loss: 1670